In [11]:
from taipy import Gui
import requests
import pandas as pd
import datetime
import numpy as np
import random
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import squarify
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: failed to find libmagic.  Check your installation

#### Airtable - fetch data

In [7]:
def connect_airtable():
    api_key = 'patngX23FWWWnl2iH.e5c8bc51522c7d6db26f4668e80775aa7f344107ec6a1204a5b388427a6e1da7'

    url = 'https://api.airtable.com/v0/appfjmrH6l5m4Z82K/tbly4c5vUFLAP91ga'

    headers = {'Authorization': f'Bearer {api_key}'}

    #request to airtable
    response = requests.get(url, headers=headers)
    data = response.json()
    records = []
    
    for record in data['records']:
        fields = record['fields']
        records.append(fields)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 9999)
    df = pd.DataFrame(records)
    return df


In [8]:
# Parameters for retrieving the airtable data
def get_data():
    #df = pd.read_csv('orders.csv')
    df = connect_airtable()
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    #data preproc
    #df['Project name (from Orders)'].fillna(df['Project name'], inplace=True)
    df['Project name (from Orders)'] = df['Project name']
    df['Name (from Customer)'] = df['Name (from Customer)'].astype(str).str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
    df['Num (from Customer)'] = df['Num (from Customer)'].apply(lambda x: int(''.join(filter(str.isdigit, str(x).replace('[', '').replace(']', '')))) if pd.notna(x) else np.nan)
    # df.dropna(subset=['Num (from Customer)'], inplace=True)
    df['Bottle Type'] = df['Item description (from Order items)'].apply(lambda x: re.findall(r'[a-zA-Z]+', x.split('-')[0])[0] if isinstance(x, str) and '-' in x else None)
    df['Bottle Type'].fillna('N/A', inplace=True)
    df['Amount Unpaid'] = df['Total paid'] - df['Order total']
    # #df['Balance due quartile desc'] = pd.qcut(df['Balance due'], q=4, labels=['3', '2', '1'], duplicates='drop')
    # #df['Balance due quartile desc'] = pd.qcut(df['Balance due'], q=4, labels=['4', '3', '2', '1'])
    df['Number of Purchases'] = df.groupby('Num (from Customer)')['Num (from Customer)'].transform('count')
    df['Client Type'] = np.where(df['Number of Purchases'] == 1, 'New', 'Returning')
    df['Average Purchase Amount'] = df.groupby('Num (from Customer)')['Order total'].transform('mean')
    df['Created: project name'] = df['Created: project name'].astype(str)
    df['Clean Date'] = df['Created: project name'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') if x != 'nan' else np.nan)
    # df = df.drop_duplicates(subset=['Num (from Customer)', 'Order num','Project name (from Orders)'], keep='last', inplace=True)
    # df = df.drop_duplicates(subset=['Num (from Customer)','Email (from Manager)','Order total','Created: project name','Date quote sent'], keep='last', inplace=True)
    df = df.replace(" ", np.NaN)
    # numeric_cols = df.select_dtypes(include=[np.number]).columns
    # df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    # #df = df.sort_values(by=['Num (from Customer)','Order num', 'Project name', 'Project name (from Orders)','Created: project name'], ascending=[True,True, True,True, True])
    df = df.sort_values(by=['Num (from Customer)','Order num', 'Project name','Clean Date'], ascending=[True,True, True, True])
    return df



In [9]:
fetch_airtable = get_data()

In [10]:
# def on_button_airtable(state):
#     notify(state, 'info', f'The text is: {state.text}')
#     state.text = "Button Pressed"

# def on_change(state, var_name, var_value):
#     if var_name == "text" and var_value == "Reset":
#         state.text = ""
#         return


#### Aggs view - 1st layout

In [11]:
#sales per month, per client and per project (total/balance due)
def sales_per_month():
    df = get_data()
    df['Clean Date'] = pd.to_datetime(df['Clean Date'])
    df['Month'] = df['Clean Date'].dt.strftime('%Y-%m')

    sales_by_month = df.groupby(['Month', 'Name (from Customer)', 'Num (from Customer)', 'Project name']).agg(
        Total_Sales=('Order total', 'sum'),
        Balance_Due=('Balance due', 'sum')
    ).reset_index()   
    return sales_by_month


In [12]:
sales_df = sales_per_month()

In [13]:
#aggregated view per month
def sales_sum_per_month():
    df = sales_per_month()
    sales_sum = df.groupby(['Month'])[['Total_Sales', 'Balance_Due']].sum().reset_index()
    return sales_sum

In [14]:
sales_sum = sales_sum_per_month()

In [15]:
def project_summary():
    df = get_data()
    project_summary = df.groupby(['Name (from Customer)', 'Num (from Customer)']).agg(
        Project_Count=('Project name', 'nunique'),
        Total_Quantity=('Quantity (from Order items)', 'sum'),
        Total_Order=('Order total', 'sum'),
        Total_Cost=('Balance due to suppliers', 'sum')
    ).reset_index()
    project_summary = project_summary.sort_values(
        by=['Project_Count', 'Total_Order', 'Total_Cost', 'Total_Quantity'],
        ascending=[False, False, False, False]
    )
    
    return project_summary

In [16]:
contributors = project_summary()

In [17]:
layout={ "barmode": "stack" }

#### Refine selection - 2nd layout

In [18]:
def month_df():
    df = get_data()
    df['Month'] = df['Clean Date'].dt.strftime('%m-%Y')
    unique_months = df['Month'].unique()  # Get unique month-year combinations
    return unique_months

In [19]:
def customer_df():
    df = get_data()
    cust = sorted(df['Name (from Customer)'].unique())
    return cust

In [20]:
month = month_df()
cust = customer_df() 

In [21]:
def fetch_project_details(month, cust):
    df = get_data()
    df = df[df['Name (from Customer)'] == cust]
    df['Month'] = df['Clean Date'].dt.strftime('%m-%Y')
    #filter for month
    if isinstance(month, str):
        month = [month]
    if month:
        df = df[df['Month'].isin(month)]

    project_details = df.groupby(['Project name']).agg(
        Total_Sales=('Order total', 'sum'),
        Balance_Due=('Balance due', 'sum')
    ).reset_index()
    
    return project_details


In [22]:
month_lov = month_df()
cust_lov = customer_df() 
selected_month = month_lov[0]
selected_cust = cust_lov[0] 
table = fetch_project_details(selected_month, selected_cust)

#### Page - markdown

In [23]:
dv_graph_selector = ['Sales Dashboard','Client Segmentation','Cohort Analysis']
dv_graph_selected = dv_graph_selector[0]

In [24]:
root_md = """
<center><h2> Rock **KYC** </h2></center>

--------------------------------------------------------------------
<center><|navbar|></center>
"""

# Sales dashboard page
sales_dashboard_md = """
<h4> Sales Dashboard (CNY) </h4>

<|Refresh w Airtable|button|on_action=refresh_table|>

<|layout|columns=1 1 1|
<|{sales_sum}|chart|mode=lines|x=Month|y[1]=Total_Sales|y[2]=Balance_Due|line[2]=dash|color[1]=blue|>

<|Contributors|expandable|
    ...
<|{contributors}|table|>
    ...
|>

<|{contributors}|chart|type=bar|x=Name (from Customer)|y[1]=Total_Order|y[2]=Total_Cost|layout={layout}|>
|>

<|layout|columns=1 1|
Test select month and name
<|{selected_cust}|selector|lov={cust_lov}|dropdown|on_change=update_table|>
<|{selected_month}|selector|lov={month_lov}|dropdown|on_change=update_table|>

Render projects for <|{selected_cust}|> on <|{selected_month}|>   
<|{table}|table|show_all|>

<|{table}|chart|>
|>
"""

# Client segmentation page
client_segmentation_md = """
<h4> Client segmentation by behaviour & transaction </h4>

<|layout|columns = 1 5|
<h6>Search for a client (enter name):  </h6>
<|{selected_cust}|input|> 

|>
"""

In [25]:
# Cohort analysis page
cohort_analysis_md = """
<h4> Client cohort (behaviour evolution) analysis  </h4>
"""



In [26]:
# Callbacks
def refresh_table(state):
    state.fetch_airtable = get_data()
    state.sales_df = sales_per_month()
    state.contributors = project_summary()
    state.month_lov = month_df()
    state.cust_lov = customer_df()
    state.selected_month = state.month_lov[0]
    state.selected_cust = state.cust_lov[0]
    state.table = fetch_project_details(state.selected_month, state.selected_cust)


In [27]:
def update_table(state):
    state.table = fetch_project_details(state.selected_month, state.selected_cust)


In [28]:
# Putting pages in a dict
pages = {
    '/': root_md,
    'Sales-Dashboard': sales_dashboard_md,
    'Client-Segmentation': client_segmentation_md,
    'Cohort-Analysis': cohort_analysis_md
}

In [ ]:
# Run Taipy GUI
gui = Gui(pages=pages)
gui.run(dark_mode=False)
     

# STOP

#### Next page on client segmentation

In [ ]:
get_data().head(2)

In [ ]:
def create_rfm_table():
    df = get_data()
    NOW = df['Clean Date'].max() + timedelta(days=1)
    period = 365
    df['Days since last order'] = df['Clean Date'].apply(lambda x: (NOW - x).days)
    aggr = {
        'Days since last order': lambda x: x.min(),  # the number of days since last order (Recency)
        'Clean Date': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)]), # the total number of orders in the last period (Frequency)
    }
    rfm = df.groupby('Num (from Customer)').agg(aggr).reset_index()
    rfm.rename(columns={'Days since last order': 'Recency', 'Clean Date': 'Frequency'}, inplace=True)
    rfm['Monetary'] = rfm['Num (from Customer)'].apply(lambda x: df[(df['Num (from Customer)'] == x) & \
                                                               (df['Clean Date'] >= NOW - timedelta(days=period))]\
                                                               ['Order total'].sum())
    quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()
    #assign ranks whereby higher monet and higher freq are better but recency smaller better
    def r_score(x):
        if x <= quintiles['Recency'][.2]:
            return 5
        elif x <= quintiles['Recency'][.4]:
            return 4
        elif x <= quintiles['Recency'][.6]:
            return 3
        elif x <= quintiles['Recency'][.8]:
            return 2
        else:
            return 1
    
    def fm_score(x, c):
        if x <= quintiles[c][.2]:
            return 1
        elif x <= quintiles[c][.4]:
            return 2
        elif x <= quintiles[c][.6]:
            return 3
        elif x <= quintiles[c][.8]:
            return 4
        else:
            return 5    
    #apply for each cust
    rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
    rfm['F'] = rfm['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
    rfm['M'] = rfm['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))
    rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
    # Function to map RFM scores to segments
    rfm['RFM Score'] = rfm['RFM Score'].astype(str).astype(int)
    def map_segment(rfm_score):
        if rfm_score in [111, 112, 121, 131, 141, 151]:
            return 'Lost'
        elif rfm_score in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
            return 'Hibernating'
        elif rfm_score in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
            return 'At Risk'
        elif rfm_score in [331, 321, 312, 221, 213]:
            return 'About To Sleep'
        elif rfm_score in [155, 154, 144, 214, 215, 115, 114, 113]:
            return 'Cannot Lose Them'
        elif rfm_score in [535, 534, 443, 434, 343, 334, 325, 324]:
            return 'Need Attention'
        elif rfm_score in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]:
            return 'Promising'
        elif rfm_score in [512, 511, 422, 421, 412, 411, 311]:
            return 'New Customer'
        elif rfm_score in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
            return 'Potential Loyalist'
        elif rfm_score in [543, 444, 435, 355, 354, 345, 344, 335]:
            return 'Loyal Customer'
        elif rfm_score in [555, 554, 544, 545, 454, 455, 445]:
            return 'Champion'
        else:
            return 'Unknown'
    
    # Apply the mapping function to create the 'Segment' column
    rfm['Segment'] = rfm['RFM Score'].apply(map_segment)
    # Dictionary to map segments to behavior and potential strategy
    segment_mapping = {
        'Champion': {
            'Segment Behavior': 'Completed a recent purchase. They buy frequently and spend the most. Reward them. Can be early adopters of new products. They act as brand ambassadors.',
            'Segment Potential Strategy': 'Reward them. Can be early adopters of new products. They act as brand ambassadors.'
        },
        'Loyal Customer': {
            'Segment Behavior': 'Buy on a regular basis. Responsive to promotions. Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.',
            'Segment Potential Strategy': 'Upsell. Offer high-value items to them. Ask them for reviews. Build loyalty.'
        },
        'Potential Loyalist': {
            'Segment Behavior': 'They recently spent a fair amount of money on the site more than once. Invite them to join a loyalty program or to get a membership. Recommend other products to them.',
            'Segment Potential Strategy': 'Invite them to join a loyalty program or to get a membership. Recommend other products to them.'
        },
        'Promising': {
            'Segment Behavior': 'Bought most recently, but not often enough for us to understand their behavior. Support them on their onboarding process. Get their satisfaction. Start to build loyalty.',
            'Segment Potential Strategy': 'Support them on their onboarding process. Get their satisfaction. Start to build loyalty.'
        },
        'New Customer': {
            'Segment Behavior': 'Completed a recent purchase, but didn’t spend a lot. Create brand awareness. Offer discounts, gifts, and free trials.',
            'Segment Potential Strategy': 'Create brand awareness. Offer discounts, gifts, and free trials.'
        },
        'Need Attention': {
            'Segment Behavior': 'Above average in recency, frequency, and monetary value, but no recent purchases. Launch limited-time offers and recommendations based on previous orders. Reactivate.',
            'Segment Potential Strategy': 'Launch limited-time offers and recommendations based on previous orders. Reactivate.'
        },
        'Cannot Lose Them': {
            'Segment Behavior': 'Completed huge, frequent purchases long ago but haven’t come back since then. Get them back with new product launches. Don’t let competitors take them. Talk to them.',
            'Segment Potential Strategy': 'Get them back with new product launches. Don’t let competitors take them. Talk to them.'
        },
        'About To Sleep': {
            'Segment Behavior': 'Below average in recency, frequency, and monetary value. At risk of losing them if you don’t reactivate. Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.',
            'Segment Potential Strategy': 'Share useful information. Recommend popular products to them. Offer new discounts. Reconnect.'
        },
        'At Risk': {
            'Segment Behavior': 'Purchased often but a long time ago. Need to bring them back! Send personalized emails to reconnect with special offers. Share useful resources with them.',
            'Segment Potential Strategy': 'Send personalized emails to reconnect with special offers. Share useful resources with them.'
        },
        'Hibernating': {
            'Segment Behavior': 'Their last purchase was long ago. The number of orders was small and the amounts spent low. Offer relevant products and special discounts. Create brand value again.',
            'Segment Potential Strategy': 'Offer relevant products and special discounts. Create brand value again.'
        },
        'Lost': {
            'Segment Behavior': 'Lowest scores in recency, frequency, and monetary value. Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.',
            'Segment Potential Strategy': 'Try to reactivate them with personalized campaigns. If it doesn’t work, ignore them.'
        }
    }
    
    # Map the segments to behavior and potential strategy
    rfm['Segment Behavior'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Behavior'])
    rfm['Segment Potential Strategy'] = rfm['Segment'].map(lambda x: segment_mapping[x]['Segment Potential Strategy'])
    return rfm

In [ ]:
create_rfm_table().head(2)

In [ ]:
def rfm_heatmap():
    rfm = create_rfm_table()
    # Calculate average values for each RFM_Level, and return a size of each segment 
    rfm_avg = rfm.groupby('Segment').agg({
        'R': lambda x: x.mode().iloc[0],
        'F': lambda x: x.mode().iloc[0],
        'M': lambda x: x.mode().iloc[0],
        'Num (from Customer)': 'count'
    }).round(1)
    rfm_avg.rename(columns={'Num (from Customer)': 'Count'}, inplace=True)
    # # Create our plot and resize it.
    # fig, ax = plt.subplots(figsize=(16, 9))
    
    # # Use squarify to create the treemap
    # sizes = rfm_avg['Count']
    # labels = [f'{label} ({count})' for label, count in zip(rfm_avg.index, sizes)]
    # #colors = plt.cm.Paired(range(len(sizes)))  
    # colors = sns.color_palette("coolwarm", len(sizes))
    # #colors = sns.diverging_palette(250, 30, l=65, center="silver",len(sizes))

    # squarify.plot(sizes=sizes, label=labels, alpha=0.6, color=colors)
    
    # # Add title and axis labels
    # plt.title("RFM Segments")
    
    # return plt.show()
    return rfm_avg.reset_index()

In [ ]:
rfm_heatmap()

In [ ]:
# def rfm_bars():
#     rfm = create_rfm_table()
#     # count the number of customers in each segment
#     segments_counts = rfm['Segment'].value_counts().sort_values(ascending=True)
    
#     fig, ax = plt.subplots()
    
#     bars = ax.barh(range(len(segments_counts)),
#                   segments_counts,
#                   color='silver')
#     ax.set_frame_on(False)
#     ax.tick_params(left=False,
#                    bottom=False,
#                    labelbottom=False)
#     ax.set_yticks(range(len(segments_counts)))
#     ax.set_yticklabels(segments_counts.index)
    
#     for i, bar in enumerate(bars):
#             value = bar.get_width()
#             if segments_counts.index[i] in ['Champion', 'Loyal Customer']:
#                 bar.set_color('firebrick')
#             ax.text(value,
#                     bar.get_y() + bar.get_height()/2,
#                     '{:,} ({:}%)'.format(int(value),
#                                        int(value*100/segments_counts.sum())),
#                     va='center',
#                     ha='left'
#                    )
    
#     return plt.show()

In [ ]:
def rfm_bars():
    rfm = create_rfm_table()
    # count the number of customers in each segment
    segments_counts = rfm['Segment'].value_counts().sort_values(ascending=True)
    return segments_counts
    # fig, ax = plt.subplots()
    
    # bars = ax.barh(range(len(segments_counts)),
    #               segments_counts,
    #               color='silver')
    # ax.set_frame_on(False)
    # ax.tick_params(left=False,
    #                bottom=False,
    #                labelbottom=False)
    # ax.set_yticks(range(len(segments_counts)))
    # ax.set_yticklabels(segments_counts.index)
    
    # for i, bar in enumerate(bars):
    #         value = bar.get_width()
    #         if segments_counts.index[i] in ['Champion', 'Loyal Customer']:
    #             bar.set_color('firebrick')
    #         ax.text(value,
    #                 bar.get_y() + bar.get_height()/2,
    #                 '{:,} ({:}%)'.format(int(value),
    #                                    int(value*100/segments_counts.sum())),
    #                 va='center',
    #                 ha='left'
    #                )
    
    # return plt.show()

In [ ]:
# properties = {
#     segments_counts,
#     #colors=['silver' if idx not in ['Champion', 'Loyal Customer'] else 'firebrick' for idx in segments_counts.index],
#     "y":segments_counts.index,
#     "x":range(len(segments_counts),
#     "orientation":    "h",
#     "layout": {
#         # Set a relevant title for the x axis
#         "xaxis": { "title": "Counts" },
#         # Hide the legend
#         "showlegend": True
# }

In [ ]:
rfm_bars()

In [ ]:
search_client = 'WILLIBALD'

In [ ]:
def client_segment_map(search_client):
    rfm = create_rfm_table()
    df = get_data()
    df = df.merge(rfm, on='Num (from Customer)', how='left')
    return df[df['Name (from Customer)'].str.contains(search_client)][['Name (from Customer)', 'Segment','Segment Behavior','Segment Potential Strategy']]

In [ ]:
client_segment_map(search_client)

In [ ]:
# dv_graph_selector = ['Client Segmentation','Cohort Analysis']
# dv_graph_selected = dv_graph_selector[0]

In [ ]:
page =  """
<center><h2> Rock **KYC** </h2></center>

<|{dv_graph_selected}|toggle|lov={dv_graph_selector}|>

--------------------------------------------------------------------
<|part|render={dv_graph_selected == 'Client Segmentation'}|
<h4> Client segmentation by behaviour & transaction </h4>

<|layout|columns = 1 5|
<|
<h6>Search for a client (enter name):  </h6>
<|{search_client}|input|> 
|>

<|
<h6><|{search_client}|> summary and action: </h6>
<|{client_segment_map(search_client)}|table|width=80%|>
|>
|>

same row 2 columns for rfm_heatmap() rfm_bars()
<|layout|columns = 1 1|
<|
<h6>Our clients overall </h6>
<|{client_segment_map(search_client)}|chart|type=heatmap|z=Count|x=R|y=F|layout={layout}|>
|>
<| 
<h6>Our clients by size </h6>
<|{rfm_bars()}|chart|type=bar|>
|>
|>

|>

<|part|render={dv_graph_selected == 'Cohort Analysis'}|
<h4> Client cohort (behaviour evolution) analysis  </h4>



|>
"""

In [ ]:
# def on_change():
    

In [ ]:
# # Run Taipy GUI
# gui = Gui(page)
# gui.run(dark_mode=False)
     